In [1]:
%cd ..
from src.common import *
%cd -
import boto3
from botocore.errorfactory import ClientError
import json
import pandas as pd
s3 = boto3.resource('s3')

/home/ec2-user/SageMaker/data-science-product-image
/home/ec2-user/SageMaker/data-science-product-image/notebooks


In [7]:
manifest = []
with open("input.manifest") as file:
    for line in file:
        manifest.append(json.loads(line.strip(), strict=False))

In [8]:
manifest[1]

{'product_id': 125491771,
 'source_image_url': 'https://images.rewardstyle.com/img?v=1&width=256&height=256&crop&p=125491771',
 'target_image_url': 's3://data-science-labeling-input/product-insight/new-category-20k/images/125491771.jpg',
 'source-ref': 's3://data-science-labeling-input/product-insight/new-category-20k/images/125491771.jpg',
 'title': 'Fit 1 Chinos',
 'description': 'Free shipping and returns on rag & bone Fit 1 Chinos at Nordstrom.com. <p>An extra-slim fit defines the modern look of classic four-pocket chinos crafted from stretch-woven Japanese cotton.</p>'}

In [3]:
cache = {}
for product in manifest:
    cache[product['target_image_url']] = product

NameError: name 'manifest' is not defined

In [10]:
new_manifest = []
bucket = s3.Bucket('data-science-labeling-input')

for image in bucket.objects.filter(Prefix="product-insight/new-category-20k/images"):
    key = "s3://data-science-labeling-input/" + image.key
    if key in cache:
        #del cache[key]
        new_manifest.append(cache[key])


In [37]:

# with open("final-input.manifest", "w") as file:
#     for p in new_manifest:
#         file.write(json.dumps(p) + "\n")

In [11]:
len(cache)

20497

In [12]:
len(manifest)

23706

In [13]:
len(new_manifest)

20372

In [14]:
#number of uniques 
unique = [x['product_id'] for x in manifest]

import numpy as np
len(np.unique(unique))

20497

In [15]:
#replace NaNs with empty string
df = pd.DataFrame(new_manifest).fillna('')
df['product_id'] = df['product_id'].astype(str)
df.head(1)

,product_id,source_image_url,target_image_url,source-ref,title,description
0,100035589,https://images.rewardstyle.com/img?v=1&width=2...,s3://data-science-labeling-input/product-insig...,s3://data-science-labeling-input/product-insig...,Stretch Weekday Warrior Dress Pants,


In [16]:
new_manifest_dict = df.to_dict('records')

import json 

with open('final-input.manifest', 'w') as file:
    for product in new_manifest_dict:
        file.write(json.dumps(product) + "\n")

In [60]:
new_manifest_dict[0]

{'product_id': 100035589,
 'source_image_url': 'https://images.rewardstyle.com/img?v=1&width=256&height=256&crop&p=100035589',
 'target_image_url': 's3://data-science-labeling-input/product-insight/new-category-20k/images/100035589.jpg',
 'source-ref': 's3://data-science-labeling-input/product-insight/new-category-20k/images/100035589.jpg',
 'title': 'Stretch Weekday Warrior Dress Pants',
 'description': ''}